This script takes a price dataset and a CSV of foundation models as input, and outputs the price dataset with an indicator column showing which models are foundation models.

The gold standard list of foundation models comes from this table provided by Stanford's Center for Research on Foundation Models:

https://github.com/stanford-crfm/ecosystem-graphs/blob/main/resources/all_assets.csv (last updated on 2024-02-20)

Place it in the folder `CRFM data` so that it can be loaded below.

Then, place the price dataset in this folder with the filename `price dataset.csv` and run this script. It will produce a copy with one more boolean column, saved as `price dataset with FM indicator.csv`.

Note that some models are named differently in the Epoch database and the CRFM dataset, so you should manually check the models that are not matched as foundation models to see if there is a match with a slightly different name (often with/without a parameter count).

In [1]:
import pandas as pd
import numpy as np
from pprint import pprint

Import the CRFM foundation model ecosystem table of models, datasets, and applications, then filter to models:

In [2]:
crfm_table = pd.read_csv('CRFM data/all_assets.csv')
price_dataset = pd.read_csv('price dataset.csv')

In [3]:
crfm_models = crfm_table[crfm_table.type == 'model']
crfm_models.head()

,type,name,organization,description,created_date,url,datasheet,modality,size,sample,...,adaptation,output_space,terms_of_service,monthly_active_users,user_distribution,failures,model_card,training_emissions,training_time,training_hardware
2,model,FalconLite2,Amazon,FalconLite2 is a fine-tuned and quantized Falc...,2023-08-08,https://huggingface.co/amazon/FalconLite2,NaN,text; text,40B parameters (dense),NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,https://huggingface.co/amazon/FalconLite2,NaN,NaN,NaN
3,model,StarCoder,BigCode,StarCoder is a Large Language Model for Code (...,2023-05-09,https://arxiv.org/pdf/2305.06161.pdf,NaN,code; code,15.5B parameters (dense),NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,https://huggingface.co/bigcode/starcoder,16.68 tons of CO2eq,2 days,64 NVIDIA A100 GPUs
4,model,SantaCoder,BigCode,Multilingual code model derived from findings ...,2023-02-24,https://arxiv.org/pdf/2301.03988.pdf,NaN,code; code,1.1B parameters (dense),NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3.1 days,96 NVIDIA Tesla V100 GPUs
6,model,YaLM,Yandex,YaLM is a 100B parameter autoregressive model ...,2022-06-22,https://medium.com/yandex/yandex-publishes-yal...,NaN,text; text,100B parameters (dense),NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Yandex 800 A100 Cluster
10,model,Koala,Berkeley,A relatively small chatbot trained by fine-tun...,2023-04-03,https://bair.berkeley.edu/blog/2023/04/03/koala/,NaN,text; text,13B parameters (dense),NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,https://huggingface.co/TheBloke/koala-7B-GPTQ-...,NaN,6 hours,8 A100 GPUs


In [4]:
foundation_models = crfm_models['name'].values

The list of foundation models in the CRFM dataset:

In [5]:
foundation_models.tolist()

['FalconLite2',
 'StarCoder',
 'SantaCoder',
 'YaLM',
 'Koala',
 'Gorilla',
 'OpenLLaMA',
 'PolyCoder',
 'CodeGeeX',
 'CogView',
 'CogView 2',
 'CogVideo',
 'GLM-130B',
 'CogVLM',
 'Jurassic-1',
 'Jurassic-1 Instruct',
 'Jurassic-2',
 'Inflection-1',
 'Inflection-2',
 'GenSLM',
 'Falcon-40B',
 'Falcon-180B',
 'Mistral',
 'T5',
 'Internal Google BERT',
 'LaMDA',
 'Flan-T5',
 'UL2',
 'Parti',
 'Imagen',
 'VATT',
 'PaLM',
 'Med-PaLM',
 'Med-PaLM Multimodal',
 'MultiMedQA',
 'Flan-PaLM',
 'Flan-U-PaLM',
 'U-PaLM',
 'PaLM-SayCan',
 'GLaM',
 'MUM',
 'Phenaki',
 'Flan-UL2',
 'MusicLM',
 'SoundStream',
 'w2v-BERT',
 'MuLan',
 'MusicLM semantic model',
 'MusicLM acoustic model',
 'Noise2Music',
 'Noise2Music pseudolabeler',
 'Minerva',
 'USM',
 'PaLM-E',
 'ViT-22B',
 'AudioLM',
 'PaLI',
 'ViT-e',
 'Vid2Seq',
 'Google Joint SLM',
 'PaLM 2',
 'MedLM',
 'Gemini',
 'Skywork',
 'GPT-J',
 'GPT-Neo',
 'GPT-NeoX',
 'VQGAN-CLIP',
 'Pythia',
 'Llemma',
 'GPT-2',
 'GPT-3',
 'Codex',
 'InstructGPT',
 'Whis

Let's check Epoch's list of models to see if anything is named differently:

In [6]:
price_dataset['System'].values.tolist()

['Yuan 1.0',
 'OpenAI TI7 DOTA 1v1',
 'Claude 2',
 'Turing-NLG',
 'GPT-3.5 (text-davinci-003)',
 'AlphaGo Master',
 'Minerva (540B)',
 'FTW',
 'AlphaGo Zero',
 'NASv3 (CIFAR-10)',
 'AlphaZero',
 'PaLM (540B)',
 'OPT-175B',
 'ALIGN',
 'AlphaGo Fan',
 'GPT-4',
 'GLaM',
 'AlphaGo Lee',
 'AlphaStar',
 'OpenAI Five',
 'RoBERTa Large',
 'OpenAI Five Rerun',
 'GNMT',
 'GShard (dense)',
 'AlphaCode',
 'Megatron-LM (8.3B)',
 'Gopher (280B)',
 'Meena',
 'IMPALA',
 'PaLM 2',
 'Megatron-BERT',
 'Falcon 180B',
 'JFT',
 'AmoebaNet-A (F=448)',
 'Megatron-Turing NLG 530B',
 'T5-3B',
 'LaMDA',
 'T5-11B',
 'BigGAN-deep 512x512',
 'ResNeXt-101 32x48d',
 'Chinchilla',
 'ERNIE 3.0 Titan',
 'LLaMA-65B',
 'DALL-E',
 'GPT-2 (1.5B)',
 'GOAT',
 'Switch',
 'iGPT-XL',
 'Parti',
 'mT5-XXL',
 'ByT5-XXL',
 'BlenderBot 3',
 'ChatGLM3',
 'Yi-34B',
 'Qwen-72B',
 'Gemini Ultra',
 'ProtT5-XXL',
 'GPT-3 175B (davinci)',
 'Inflection-2',
 'BERT-Large',
 'Meta Pseudo Labels',
 'Llama 2-70B',
 'U-PaLM (540B)',
 'HyperClova',

Add models with not-exactly-matching names to the foundation model list:

In [7]:
foundation_models = np.append(
    foundation_models,
    ['GPT-3 175B (davinci)', 'ChatGLM3', 'HyperClova', 'Megatron-Turing NLG 530B', 'Minerva (540B)', 'PaLM (540B)']
)

Filter the price dataset to only include frontier models listed by CRFM:

In [8]:
is_foundation_model = [_ in foundation_models for _ in price_dataset['System']]

In [9]:
foundation_model_prices = price_dataset[is_foundation_model]

In [10]:
foundation_model_prices.to_csv('foundation model prices.csv')

In [11]:
price_dataset['Foundation model?'] = is_foundation_model

In [12]:
price_dataset.to_csv('price dataset with FM indicator.csv')